In [13]:
# import pandas as pd
# import spacy
# import string
# import regex as re
# import nltk
# import ast
# import copy
# import glob
# import advertools as adv
# import plotly.graph_objects as go
# import time

# pd.options.display.max_colwidth = 285



# from textblob import TextBlob

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# from nltk.corpus import stopwords
# from urllib.parse import urlparse
# from textblob import Word
# from sklearn.metrics.pairwise import cosine_similarity


# nlp = spacy.load("en_core_web_sm")



# stemmer = nltk.SnowballStemmer("english")


# # nltk.download('wordnet')
# # nltk.download('omw-1.4')

## Import sentence_transformers and load pre-trained model

In [14]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/stsb-roberta-base')

2022-08-25 13:07:56,080 | INFO | CrossEncoder.py:56 | __init__ | Use pytorch device: cpu


In [15]:
def get_tweet_conversation_pair(df_conversation):

    pairs = []
    tweet_col = df_conversation['clean_tweet']
    conv_col = df_conversation['clean_text']
    for tweet, conv in zip(tweet_col, conv_col):
        pairs.append([tweet, conv])
    
    return pairs

In [16]:
def variance_conversation_semantic(sub_df):
    """
    input:
        sub-dataframe, containing responses of a tweet
    output:
        variance of semantic score of the tweet's conversation
    """
    if len(sub_df) > 1:
        return sub_df['semantic_similarity'].var()
    else:
        return 0

In [17]:
def split_dataframe(df, chunk_size = 10000): 
    list_of_chunks = []
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        list_of_chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return list_of_chunks

In [18]:
def add_tweet_conv_semantic_similarity(df_conversation, name_str):
    dfs = split_dataframe(df_conversation)
    print(f'Total Steps:{len(dfs)}')
    for i, df in enumerate(dfs):
        
        dftemp = df.reset_index()
        start_time = time.time()
        print(f'start {i}th chunk')
        pairs = get_tweet_conversation_pair(dftemp) 
        
        dftemp['semantic_similarity'] = model.predict(pairs, show_progress_bar=False)
        print("--- %s seconds ---" % (time.time() - start_time))
        
        dftemp.to_feather(f'/home/niloofar/Developement/Climate-Change/data/temp_dfs/{name_str}/rest/temp_{name_str}_{i}th_chunk.feather')
        print(f"{i}th chunk completed and stored.")
        
        